### imports

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
import sys
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.spatial import ConvexHull
from matplotlib import ticker, cm
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
from IPython.display import clear_output

from ipynb.fs.full.optimizedTIO_paper import refAccuracyScheduler

clear_output()

### dimensions & parameters

In [2]:
myseed = '2404'
np.random.seed(int(myseed))

# basic dimensions
d = int(50)
n = int(1e2)

size = 'n_'+str(n)+'_d_'+str(d)+'_'
sigma = 1e-3
mu_test = 1e-1
L_test = 2/sigma+mu_test # see Devolder, Glineur, Nesterov 2013 

# TOIO parameters
M = 100
m = 0

REPS = 5
benchmark = True

# data
O = np.random.normal(0,5,(n,d))
theta_hat = np.mean(O,0)
MAT_O = O@O.T
eigvals = np.real(np.linalg.eigvals(MAT_O))
min_eig,max_eig = max(0,min(eigvals)),max(eigvals)
min_eig/max_eig

0.0

### utils

In [3]:
"""
projection on simplex
"""
reltol_simplex,abstol_infeas_simplex = 1e-6,1e-7
def projection_simplex(vector,args=None,inequality=False):
    if args is None:
        radius = 1
    else:
        radius = args[0]
    if inequality and min(vector)>=-abstol_infeas_simplex and max(sum(np.maximum(0,vector))-radius,0)<=reltol_simplex*radius:
        return vector
    n_features = vector.shape[0]
    u = np.sort(vector)[::-1]
    cssv = np.cumsum(u) - radius
    ind = np.arange(n_features) + 1
    cond = u - cssv / ind > 0
    rho = ind[cond][-1]
    theta = cssv[cond][-1] / float(rho)
    w = np.maximum(vector - theta, 0)
    return w

In [4]:
"""
(inexact FOIO) FGD from Stonyakin et. al. (2021)

main :: R^2 in our paper

"""
def adaptive_FGD(x0,f_oracle,delta_policy,Psi=lambda u: 0, proxPsi = lambda u,g,ss: u, max_iter = int(5000), primal_target = 1e-8, main = np.inf, r_estimated = 0 ,L=1.0,m=0.0,mu=0.0,down=1.5,verbose=True,freq=10,keep=True,max_omega=int(1e6)):
    
    h_grad = lambda u: u # relative smoothness kernel's gradient
    
    # init.
    k = 0
    x,y,v = x0.copy(),x0.copy(),x0.copy()
    warm_start_pack1,warm_start_pack2 = None,None
    F_out = np.inf
    dimension = len(x)
    local_L = L*down
    L_m = local_L
    
    best_certified_value = np.inf
        
    s = mu+m
    A = 0
    accum_k = 0
    omega_count = 0
    
    last_impact = 1
    last_relative_cost = 1 
    next_impact = 1
    next_relative_cost = 1
    last_ok_delta = None
    
    power_estimate = 1/(1+r_estimated)

    # storing sequence
    infos = np.array([[0,0,0,np.inf,None]])
    
    # main algorithmic loop
    start_ = time.time()
    
    while k<max_iter:
        
        local_L = local_L/down
        local_smoothness = False
        
        while local_smoothness==False:
        
            a_plus = ((1+A*s)+((1+A*s+4*A*local_L)**(1/2) * (1+A*s)**(1/2)))/(2*local_L) # new impact

            next_impact = A+a_plus 
            next_relative_cost = 1

            accuracy_k = delta_policy(k,next_impact,last_impact,last_ok_delta,power_estimate,next_relative_cost,last_relative_cost)

            y = (A*x + a_plus*v)/(A+a_plus)

            fk,gk,warm_start_pack1,true_delta_k1,omega_k1 = f_oracle(y,'full',accuracy=accuracy_k,warm_start=warm_start_pack1) # FOI collection 1

            v = proxPsi(h_grad(v)*local_L*a_plus/(A+a_plus) + h_grad(y)*mu, gk, stepsize=(mu+local_L*a_plus/(A+a_plus))**(-1)) # PROXIMAL step 
            x = (A*x + a_plus*v)/(A+a_plus)
            
            valider,warm_start_pack2,true_delta_k2,omega_k2 = f_oracle(x,'value',accuracy=accuracy_k,warm_start=warm_start_pack2) # FOI collection 2
            
            omega_count += (omega_k1+omega_k2)
            
            slack_upper_model = fk+sum(gk*(x-y))+local_L/2*sum((x-y)**2)+accuracy_k-valider
            
            if slack_upper_model>=0:
                effective_delta_k1 = max(0,accuracy_k-slack_upper_model)
                accum_k = (A*accum_k+(A+a_plus)*(effective_delta_k1+true_delta_k2))/(A+a_plus)
                local_smoothness = True
            else:
                local_L = 2*local_L
                            
        # updates of iteration counter and accumulators
        A += a_plus  
        last_impact = A
        last_relative_cost = 1
        last_ok_delta = accuracy_k
        
        L_m = (k*L_m + local_L)/(k+1)
        
        if valider+true_delta_k2<best_certified_value:
            best_certified_value = valider+true_delta_k2
        
        certif = main/A+accum_k                        
        
        if (k % freq == 0) and k>0:
            
            if keep:
                fxk,_,true_delta,_ = f_oracle(x,'value',accuracy=1e-10,warm_start=warm_start_pack2) # FOI collection storage 
                Fxk = fxk+true_delta+Psi(x)
                infos = np.vstack((infos,np.array([[Fxk,accum_k,omega_count,best_certified_value,accuracy_k]]))) # storage time 
            else:
                Fxk = valider+true_delta_k2+Psi(x)
            if verbose:
                print("k = %i, value = %e, totError = %e, certif. = %e, L m = %e" %
                      (k,Fxk,accum_k,certif,L_m))
                print(' ')
        
        k += 1
        
        ## check for accuracy
        if certif <= primal_target or omega_count>max_omega:
            
            if verbose:
                if omega_count<=max_omega:
                    print(' => maximum # inner-iterations reached: primal target not certified <= ')
                else:
                    print(' => early convergence: primal target certified <= ')
            if keep: 
                return x,np.array(infos[1:])
            else:
                return x,omega_count,best_certified_value
    if keep:            
        
        return x,np.array(infos[1:])
    
    else:
        
        return x,omega_count,best_certified_value

In [5]:
"""
momentum for inner problem 
"""
if min_eig*sigma>0:
    r = 0
    def momentum(theta):
        return 1/(1+2*np.sqrt(min_eig/max_eig)),theta
else:
    r = 1/2
    def momentum(theta):
        new_theta = 1/2*(theta*np.sqrt(theta**2+4)-theta**2)
        inertia = new_theta*(1/theta-1)
        return inertia,new_theta

In [6]:

my_proj = lambda u: projection_simplex(u)

def myPsi(x):
    return 0

def myProxPsi(anchor, direction, stepsize):
    return projection_simplex(-stepsize*(direction-anchor))

def myLMM(anchor,direction,mu=0.0):
    id_min = np.argmin(direction)
    buf = np.zeros(len(anchor))
    if mu==0:
        buf[id_min] = 1
        return buf # for radius 1
    else:
        return projection_simplex(anchor-direction*mu**(-1))

In [7]:
### oracle preparation
def RLP_oracle(x,mode='value',accuracy=1e-3,warm_start=None,max_inner=int(5e4),verb=False):
    
    ## implementation of FISTA with FW-GAP and FO worst-case bound monitoring
    if warm_start is not None:
        inner_seq1,inner_seq1_lag,inner_seq2 = warm_start.copy(),warm_start.copy(),warm_start.copy()
    else:
        inner_seq1,inner_seq1_lag,inner_seq2 = np.ones(n)/n,np.ones(n)/n,np.ones(n)/n
        
    k_inner,cond = 0,False
    
    stepsize = 1/(sigma*max_eig)
    local_strong = sigma*min_eig
 
    lower_bound = -np.inf
    inertia,theta = 0,1
    grad_shift = (x+sigma*theta_hat)@O.T
    
    while k_inner<max_inner and cond==False:
        # proximal gradient step
        inner_seq1_lag = inner_seq1
        grad_inner_seq2 = sigma*MAT_O@inner_seq2-grad_shift
        inner_seq1 = projection_simplex(inner_seq2-stepsize*grad_inner_seq2)
        # momentum 
        inertia,theta = momentum(theta) 
        inner_seq2 = inner_seq1 + inertia*(inner_seq1-inner_seq1_lag)
        # FW check
        grad_inner_seq1 = sigma*MAT_O@inner_seq1-grad_shift
        buf = np.zeros(n)
        buf[np.argmin(grad_inner_seq1)] = 1
        val_inner_seq1 = sigma/2*sum((inner_seq1@O-theta_hat)**2) - sum((inner_seq1@O)*x)
        lower_bound = max(lower_bound,sum(grad_inner_seq1*(buf-inner_seq1))+val_inner_seq1)
        acc_cert = min(4/stepsize/(k_inner+1)**2,2/stepsize*(1-np.sqrt(stepsize*local_strong))**(k_inner),val_inner_seq1-lower_bound)
        cond = acc_cert<=accuracy/2
        if verb and k_inner%10==0:
            print('inner it -> '+str(k_inner+1)+' | DUALITY GAP: '+str(val_inner_seq1-lower_bound))
        # inner iteration counter increase
        k_inner+=1
        
    # evaluation time
    if mode=='value':
        return -val_inner_seq1 + mu_test/2*sum(x**2),inner_seq1,acc_cert*2,k_inner
    elif mode=='gradient':
        return inner_seq1@O + mu_test*x,inner_seq1,acc_cert*2,k_inner
    else:
        return -val_inner_seq1 + mu_test/2*sum(x**2), inner_seq1@O+mu_test*x,inner_seq1,acc_cert*2,k_inner

### `experiment` 

multiple variations of $\bar{\delta}$ and $N_r$ in <i>offline</i> schedules

In [8]:
Nr_list = [50]
if r==0:
    bar_delta_list = [9e-3,1e-4,1e-6]
else:
    bar_delta_list = [1e-1,1e-3,1e-5]

In [9]:
### benchmark to refine F_star
x0 = my_proj(np.random.uniform(-2,2,d))
if r==0:
    pt = 1e-10
    x_benchmark,_,_ = \
               adaptive_FGD(x0,f_oracle=lambda x,mode,accuracy,warm_start: RLP_oracle(x,mode,accuracy,warm_start=warm_start),delta_policy=lambda k, nexti,lasti,lastd,power,next_relative_cost,last_relative_cost: 1e-12,Psi = myPsi, proxPsi = myProxPsi, max_iter = int(3000), primal_target = pt, main = 2, r_estimated = r , L=L_test,m=0.0,mu=mu_test,verbose=True,freq=10,keep=False,max_omega=int(3e6))
else:
    pt = 1e-7
    x_benchmark,_,_ = \
               adaptive_FGD(x0,f_oracle=lambda x,mode,accuracy,warm_start: RLP_oracle(x,mode,accuracy,warm_start=warm_start),delta_policy=lambda k, nexti,lasti,lastd,power,next_relative_cost,last_relative_cost: 1e-8,Psi = myPsi, proxPsi = myProxPsi, max_iter = int(3000), primal_target = pt, main = 2, r_estimated = r , L=L_test,m=0.0,mu=mu_test,verbose=True,freq=10,keep=False,max_omega=int(3e6))
val_benchmark,grad_benchmark,_,error_benchmark,_ = RLP_oracle(x_benchmark,mode='full',accuracy=1e-10,max_inner=int(5e4),verb=False)
minimizer_lb_benchmark = myLMM(x_benchmark,grad_benchmark,mu_test)
lb_benchmark = val_benchmark + sum(grad_benchmark*(minimizer_lb_benchmark-x_benchmark))+ mu_test/2*sum((minimizer_lb_benchmark-x_benchmark)**2)

k = 10, value = 8.345632e-01, totError = 3.142088e-08, certif. = 3.318188e+01, L m = 8.293650e+02
 
k = 20, value = 7.322758e-01, totError = 6.133252e-08, certif. = 1.194136e+01, L m = 9.522652e+02
 
k = 30, value = 7.200075e-01, totError = 8.472943e-08, certif. = 5.916465e+00, L m = 9.608481e+02
 
k = 40, value = 7.170162e-01, totError = 1.125790e-07, certif. = 3.426836e+00, L m = 9.406328e+02
 
k = 50, value = 7.160270e-01, totError = 1.374549e-07, certif. = 2.295638e+00, L m = 9.647571e+02
 
k = 60, value = 7.156983e-01, totError = 1.633305e-07, certif. = 1.631241e+00, L m = 9.726521e+02
 
k = 70, value = 7.155503e-01, totError = 2.037775e-07, certif. = 1.190461e+00, L m = 9.711520e+02
 
k = 80, value = 7.154665e-01, totError = 2.400341e-07, certif. = 9.062499e-01, L m = 9.739826e+02
 
k = 90, value = 7.154177e-01, totError = 2.820797e-07, certif. = 7.120844e-01, L m = 9.797359e+02
 
k = 100, value = 7.153941e-01, totError = 3.221412e-07, certif. = 5.650965e-01, L m = 9.747232e+02
 

k = 830, value = 7.153365e-01, totError = 9.328770e-07, certif. = 7.560057e-05, L m = 9.475470e+02
 
k = 840, value = 7.153365e-01, totError = 9.418983e-07, certif. = 6.830941e-05, L m = 9.486140e+02
 
k = 850, value = 7.153365e-01, totError = 9.492912e-07, certif. = 6.137193e-05, L m = 9.486328e+02
 
k = 860, value = 7.153365e-01, totError = 9.337181e-07, certif. = 5.425757e-05, L m = 9.467092e+02
 
k = 870, value = 7.153383e-01, totError = 9.185573e-07, certif. = 4.815446e-05, L m = 9.461003e+02
 
k = 880, value = 7.153365e-01, totError = 9.033922e-07, certif. = 4.353263e-05, L m = 9.470226e+02
 
k = 890, value = 7.153365e-01, totError = 8.916540e-07, certif. = 3.906100e-05, L m = 9.476174e+02
 
k = 900, value = 7.153365e-01, totError = 8.962990e-07, certif. = 3.518863e-05, L m = 9.480523e+02
 
k = 910, value = 7.153365e-01, totError = 9.068702e-07, certif. = 3.178135e-05, L m = 9.488513e+02
 
k = 920, value = 7.153365e-01, totError = 9.064528e-07, certif. = 2.855563e-05, L m = 9.483

k = 1640, value = 7.153365e-01, totError = 8.924674e-07, certif. = 9.015221e-07, L m = 9.490381e+02
 
k = 1650, value = 7.153365e-01, totError = 8.905249e-07, certif. = 8.986551e-07, L m = 9.490977e+02
 
k = 1660, value = 7.153365e-01, totError = 8.908292e-07, certif. = 8.981042e-07, L m = 9.493186e+02
 
k = 1670, value = 7.153365e-01, totError = 8.942732e-07, certif. = 9.007964e-07, L m = 9.496376e+02
 
k = 1680, value = 7.153365e-01, totError = 8.947919e-07, certif. = 9.006396e-07, L m = 9.494788e+02
 
k = 1690, value = 7.153365e-01, totError = 9.007345e-07, certif. = 9.059602e-07, L m = 9.494331e+02
 
k = 1700, value = 7.153365e-01, totError = 9.148234e-07, certif. = 9.195019e-07, L m = 9.495657e+02
 
k = 1710, value = 7.153365e-01, totError = 9.248172e-07, certif. = 9.290267e-07, L m = 9.503998e+02
 
k = 1720, value = 7.153365e-01, totError = 9.206901e-07, certif. = 9.244587e-07, L m = 9.504946e+02
 
k = 1730, value = 7.153365e-01, totError = 9.122026e-07, certif. = 9.155831e-07, L

k = 2450, value = 7.153365e-01, totError = 8.227339e-07, certif. = 8.227349e-07, L m = 9.481092e+02
 
k = 2460, value = 7.153365e-01, totError = 8.230661e-07, certif. = 8.230670e-07, L m = 9.482833e+02
 
k = 2470, value = 7.153365e-01, totError = 8.269749e-07, certif. = 8.269758e-07, L m = 9.480354e+02
 
k = 2480, value = 7.153365e-01, totError = 8.532696e-07, certif. = 8.532704e-07, L m = 9.479808e+02
 
k = 2490, value = 7.153365e-01, totError = 8.665143e-07, certif. = 8.665150e-07, L m = 9.481359e+02
 
k = 2500, value = 7.153365e-01, totError = 8.502004e-07, certif. = 8.502010e-07, L m = 9.478710e+02
 
k = 2510, value = 7.153365e-01, totError = 8.526721e-07, certif. = 8.526726e-07, L m = 9.477166e+02
 
k = 2520, value = 7.153367e-01, totError = 8.504998e-07, certif. = 8.505003e-07, L m = 9.474560e+02
 
k = 2530, value = 7.153365e-01, totError = 8.566020e-07, certif. = 8.566024e-07, L m = 9.472755e+02
 
k = 2540, value = 7.153365e-01, totError = 8.724014e-07, certif. = 8.724018e-07, L

In [10]:
minimizer_lb_benchmark = myLMM(x_benchmark,grad_benchmark,mu_test)
lb_benchmark = val_benchmark + sum(grad_benchmark*(minimizer_lb_benchmark-x_benchmark))+ mu_test/2*sum((minimizer_lb_benchmark-x_benchmark)**2)

In [11]:
mu_test/L_test

4.9997500124993755e-05

In [12]:
F_star = lb_benchmark
lb_benchmark

0.7153351389574667

In [ ]:
condition = np.format_float_scientific(mu_test/L_test,3)

for N_ref in Nr_list:
    for bar_delta in bar_delta_list:
            
        accuracy_ref = refAccuracyScheduler(N_ref,M,bar_delta,r,arguments=[mu_test,0.0,L_test],typ='FGD',verbose=False)
                
        def delta_hat_constant(k,next_impact,last_impact,last_ok_delta,power,next_relative_cost,last_relative_cost):

            return bar_delta

        def delta_hat(k,next_impact,last_impact,last_ok_delta,power,next_relative_cost,last_relative_cost):

            if k<N_ref:

                return accuracy_ref[k]

            return  max(m*bar_delta,min(M*bar_delta,last_ok_delta*(last_impact/last_relative_cost * next_relative_cost/next_impact)**power))

            
        print('new scenario || N = '+str(N_ref)+', bar_delta = '+str(bar_delta)+', r = '+str(r))
        print(' ')
        
        stock_constant,stock_tunable,stock_poly3 = None,None,None
        if mu_test>0:
            stock_linear = None

        for i_rep in range(REPS):
            
            x0 = my_proj(np.random.uniform(0,1,d))

            print('rep#'+str(i_rep))
            print(' ')
                        
            # constant phase
            x_constant,infos_constant = \
               adaptive_FGD(x0,f_oracle=lambda x,mode,accuracy,warm_start: RLP_oracle(x,mode,accuracy,warm_start=warm_start),delta_policy=delta_hat_constant,Psi = myPsi, proxPsi = myProxPsi, max_iter = int(1000), primal_target = pt/100, main = 2, r_estimated = r , L=L_test,m=0.0,mu=mu_test,verbose=False,freq=10,keep=True)
            mini_constant = np.argmin(infos_constant[:,0])
            GAP_constant = infos_constant[mini_constant,0]-F_star
            certif_GAP_constant = infos_constant[3,-1] - F_star
            print('constant: '+str(np.format_float_scientific(GAP_constant,3))+' for '+\
                  str(infos_constant[mini_constant,2])+' inner-iterations')
            
            infos_constant[:,0] = infos_constant[:,0]-F_star
            
            if stock_constant is None:
                stock_constant = infos_constant
            else:
                stock_constant = np.vstack((stock_constant,infos_constant))
            
            # tunable phase
            x_tunable,infos_tunable = \
               adaptive_FGD(x0,f_oracle=lambda x,mode,accuracy,warm_start: RLP_oracle(x,mode,accuracy,warm_start=warm_start),delta_policy=delta_hat,Psi = myPsi, proxPsi = myProxPsi, max_iter = int(1000), primal_target = pt/100, main = 2, r_estimated = r , L=L_test,m=0.0,mu=mu_test,verbose=False,freq=10,keep=True)
            mini_tunable = np.argmin(infos_tunable[:,0])
            GAP_tunable = infos_tunable[mini_tunable,0]-F_star
            certif_GAP_tunable = infos_tunable[3,-1] - F_star
            print('tunable: '+str(np.format_float_scientific(GAP_tunable,3))+' for '+\
                  str(infos_tunable[mini_tunable,2])+' inner-iterations')
            
            infos_tunable[:,0] = infos_tunable[:,0]-F_star
            
            if stock_tunable is None:
                stock_tunable = infos_tunable
            else:
                stock_tunable = np.vstack((stock_tunable,infos_tunable))
            
            # poly3 phase
            x_poly3,infos_poly3 = \
               adaptive_FGD(x0,f_oracle=lambda x,mode,accuracy,warm_start: RLP_oracle(x,mode,accuracy,warm_start=warm_start),delta_policy=lambda k, nexti,lasti,lastd,power,next_relative_cost,last_relative_cost: bar_delta/(k+1)**3,Psi = myPsi, proxPsi = myProxPsi, max_iter = int(1000), primal_target = pt/100, main = 2, r_estimated = r , L=L_test,m=0.0,mu=mu_test,verbose=False,freq=10,keep=True)
            mini_poly3 = np.argmin(infos_poly3[:,0])
            GAP_poly3 = infos_poly3[mini_poly3,0]-F_star
            certif_GAP_poly3 = infos_poly3[3,-1] - F_star
            print('poly3: '+str(np.format_float_scientific(GAP_poly3,3))+' for '+\
                  str(infos_poly3[mini_poly3,2])+' inner-iterations')
            
            infos_poly3[:,0] = infos_poly3[:,0] - F_star
                    
            if stock_poly3 is None:
                stock_poly3 = infos_poly3
            else:
                stock_poly3 = np.vstack((stock_poly3,infos_poly3))
            
            if mu_test>0:
                
                # linear phase
                x_linear,infos_linear = \
                   adaptive_FGD(x0,f_oracle=lambda x,mode,accuracy,warm_start: RLP_oracle(x,mode,accuracy,warm_start=warm_start),delta_policy=lambda k, nexti,lasti,lastd,power,next_relative_cost,last_relative_cost: bar_delta*(1-np.sqrt(mu_test/L_test))**(k),Psi = myPsi, proxPsi = myProxPsi, max_iter = int(1000), primal_target = pt/100, main = 2, r_estimated = r , L=L_test,m=0.0,mu=mu_test,verbose=False,freq=10,keep=True)
                mini_linear = np.argmin(infos_linear[:,0])
                GAP_linear = infos_linear[mini_linear,0]-F_star
                certif_GAP_constant = infos_linear[3,-1] - F_star
                print('linear: '+str(np.format_float_scientific(GAP_linear,3))+' for '+\
                      str(infos_linear[mini_linear,2])+' inner-iterations')
                
                infos_linear[:,0] = infos_linear[:,0] - F_star
                
                if stock_linear is None:
                    stock_linear = infos_linear
                else:
                    stock_linear = np.vstack((stock_linear,infos_linear))             
            
            print(' ')
                
        if mu_test>0:
            np.save(size+'r_'+str(r)+'_reg_online_linear_'+str(bar_delta)+'.npy',stock_linear)
            np.save(size+'r_'+str(r)+'_reg_online_constant_'+str(bar_delta)+'.npy',stock_constant)
            np.save(size+'r_'+str(r)+'_reg_online_tunable_'+str(bar_delta)+'.npy',stock_tunable)
            np.save(size+'r_'+str(r)+'_reg_online_poly3_'+str(bar_delta)+'.npy',stock_poly3)
        else:
            np.save(size+'r_'+str(r)+'_no_reg_online_constant_'+str(bar_delta)+'.npy',stock_constant)
            np.save(size+'r_'+str(r)+'_no_reg_online_tunable_'+str(bar_delta)+'.npy',stock_tunable)
            np.save(size+'r_'+str(r)+'_no_reg_online_poly3_'+str(bar_delta)+'.npy',stock_poly3)
        

        print(' ')
        print(' =========== ')
        print(' ')